# Regression

In the last lecture, we looked at the problem of classification where the goal was to find a rule that best predicts class membership from a discrete set of classes, such as "Fraud v.s. Not Fraud" or "Survived or Not Survived". In this lecture, we are going to study real-valued prediction---like predict the number of sales on a given day or predict the energy usage of a device. However, the basic construct will still be the same: you have a set of features X and a set of labels Y. The goal is to find a functional relationship between X and Y that best predicts Y.

## Example
Bike-sharing rental process is highly correlated to the environmental and seasonal settings. For instance, weather conditions, precipitation, day of week, season, hour of the day, etc. can affect the rental behaviors. The core data set is related to   the two-year historical log corresponding to years 2011 and 2012 from Capital Bikeshare system, Washington D.C., USA which is  publicly available in http://capitalbikeshare.com/system-data. 

In [82]:
import pandas as pd
data = pd.read_csv('hour.csv')
data[:5]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Let's break down each of the columns, so we have a better idea of what we are dealing with.

In [26]:
print(data.groupby(['yr'])['yr'].count()) #Two years of data

yr
0    8645
1    8734
Name: yr, dtype: int64


In [27]:
print(data.groupby(['season'])['season'].count()) #seasons count 1-4

season
1    4242
2    4409
3    4496
4    4232
Name: season, dtype: int64


In [28]:
print(data.groupby(['weekday'])['weekday'].count()) #day of the week

weekday
0    2502
1    2479
2    2453
3    2475
4    2471
5    2487
6    2512
Name: weekday, dtype: int64


Let's try something simple first and then refine our approach. Like before, we can construct features and labels.

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

def linmodel():
    feature_cols = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed']
    X = data[feature_cols].to_numpy()
    Y = data['cnt'].to_numpy()

    #create a testing dataset

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

    lin = LinearRegression()
    lin.fit(X_train,Y_train)
    Y_pred = lin.predict(X_test)

    print(r2_score(Y_test, Y_pred))

In [84]:
linmodel()

0.3975373993285606


This is better than simply picking the mean value each time. But can we do even better?

## Feature Engineering
What might be broken. Many datasets that are collected overtime exhibit seasonal effects where the data repeats:

In [31]:
print(data.groupby(['mnth'])['cnt'].mean())

mnth
1      94.424773
2     112.865026
3     155.410726
4     187.260960
5     222.907258
6     240.515278
7     231.819892
8     238.097627
9     240.773138
10    222.158511
11    177.335421
12    142.303439
Name: cnt, dtype: float64


Why might this be a problem for linear regression? Fundementally this is a non-linear relationship between the month id and the value. One simple trick is to transform the data into "distance to june" rather than keep it as a month id.

In [86]:
import numpy as np
data['mnth'] = np.abs((data['mnth'] - 6))
print(data.groupby(['mnth'])['cnt'].mean())

mnth
0    240.515278
1    227.363575
2    213.010989
3    197.563918
4    169.664756
5    135.995813
6    142.303439
Name: cnt, dtype: float64


In [87]:
linmodel()

0.39938199269945185


What else could be challenging? There could be outliers or points that are not really well correlated with the features. We can fix this with a technique called ridge regression, this approach penalizes fitting heavily to a single feature.

In [91]:
from sklearn.linear_model import Ridge
def ridgemodel(alpha=1):
    feature_cols = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed']
    X = data[feature_cols].to_numpy()
    Y = data['cnt'].to_numpy()

    #create a testing dataset

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
    
    #Y_train[0:1000] = 100

    lin = Ridge(alpha=alpha)
    lin.fit(X_train,Y_train)
    Y_pred = lin.predict(X_test)

    print(r2_score(Y_test, Y_pred))

ridgemodel(1000)


0.35450371872393094


## Non-linear models
Maybe your problem is fundementally non-linear. One easy way to make a linear model more successful at non-linear relationships to to engineer your features to capture non-linearity better:

In [93]:
from sklearn.preprocessing import PolynomialFeatures
pol = PolynomialFeatures(degree=3)
X = np.array([[1,2,3]])
Xp = pol.fit_transform(X)
Xp

array([[ 1.,  1.,  2.,  3.,  1.,  2.,  3.,  4.,  6.,  9.,  1.,  2.,  3.,
         4.,  6.,  9.,  8., 12., 18., 27.]])

In [95]:
def polymodel(degree=1):
    feature_cols = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed']
    X = data[feature_cols].to_numpy()
    Y = data['cnt'].to_numpy()
    
    pol = PolynomialFeatures(degree=degree)
    X = pol.fit_transform(X)

    #create a testing dataset

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

    lin = LinearRegression()
    lin.fit(X_train,Y_train)
    Y_pred = lin.predict(X_test)

    print('Degree:',degree,r2_score(Y_test, Y_pred))
    
    #training accuracy
    Y_pred = lin.predict(X_train)
    print(r2_score(Y_train, Y_pred))
    
    

polymodel(1)
polymodel(2)
polymodel(3)
polymodel(4) #what is going on here???

Degree: 1 0.39938199269945196
0.3874777557144905
Degree: 2 0.5716738428789065
0.5501992459025082
Degree: 3 0.6526320461664733
0.6471134195070325
Degree: 4 0.6250148884188107
0.7158030590170514


sci-kit learn also has other non-linear models that might be more effective for your task. 

In [96]:
from sklearn.ensemble import RandomForestRegressor
def rfmodel():
    feature_cols = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed']
    X = data[feature_cols].to_numpy()
    Y = data['cnt'].to_numpy()
    
    #create a testing dataset

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
    
    #Y_train[0:1000] = 100

    lin = RandomForestRegressor()
    lin.fit(X_train,Y_train)
    Y_pred = lin.predict(X_test)

    print(r2_score(Y_test, Y_pred))

rfmodel()

/Users/sanjaykrishnan/Documents/cmsc21800/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9390331254966053
